In [3]:
import subprocess
import re
import os
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import pymysql.cursors
import concurrent.futures

all_sequences = []
clusters2process = 3
alignment_info = {'Amostra': [], 'Total de Sequências': [], 'Clusters': []}

for i in range(clusters2process):
    alignment_info[f'Sequências Cluster {i+1}'] = []
    alignment_info[f'% Cluster {i+1}'] = []


In [4]:
files_report = {}
for file in os.listdir('input_lib2'):
	file_path = f'./input_lib2/{file}'
	file_name = file.replace('.fa', '')
	file_name = file_name.replace('.fasta', '')
	file_name = file_name.replace('.fas', '')

	if not os.path.exists(f"output/lib2/{file_name}"):
		os.makedirs(f"output/lib2/{file_name}")
	
	
	with open(file_path, 'r') as file:
		lines = file.readlines()

	sequences = []
	seq = ''
	count = 1

	for line in lines:
		if line.startswith('>'):
			sequences.append(f'>{count}\n')
			count += 1
		else:
			sequences.append(line)

	with open(file_path, 'w') as file:
		file.write(''.join(sequences))

	sequences = SeqIO.to_dict(SeqIO.parse(file_path, "fasta"))
	total_sequences = len(sequences.keys())
	
	# subprocess.run(
	# 	f'source /home/bioinfo/miniconda3/etc/profile.d/conda.sh && \
	# 	conda activate cdhit-env && \
	# 	cd-hit -T 10 -i {file_path} -o ./output/lib2/{file_name}/clustering.fasta -n 3 -aL 0.8 -aS 0.8 -c 0.7 && \
	# 	conda deactivate',
	# 	shell=True,
	# 	executable='/bin/bash'
	# )


	with open(f'output/lib2/{file_name}/clustering.fasta.clstr', 'r') as f:
		clustering = f.read()

		clusters = re.findall(r'>Cluster \d*', clustering)
		clusters_content = re.split(r'>Cluster \d*\n', clustering)
		clusters_content.remove('')
		clusters_sizes = {}
		for pos, cluster in enumerate(clusters):
			cluster_number = int(cluster[-1])
			cluster_content = clusters_content[cluster_number]
			cluster_seq_count = len(cluster_content.split('\n')) - 1
			if cluster_seq_count >= 5:
				clusters_sizes[cluster_number] = cluster_seq_count


		clusters_sizes = {k: v for k, v in sorted(clusters_sizes.items(), reverse=True, key=lambda item: item[1])}

	files_report[file_name] = {'clusters_content': clusters_content, 'clusters_sizes': clusters_sizes, 'sequences': sequences}
	
	
	
	
	alignment_info['Amostra'].append(file_name)
	alignment_info['Total de Sequências'].append(total_sequences)
	alignment_info['Clusters'].append(len(clusters_sizes.keys()))

	for i in range(clusters2process):
		if len(list(clusters_sizes.keys())) > i:
			nseqs_cluster = clusters_sizes[list(clusters_sizes.keys())[i]]
			alignment_info[f'% Cluster {i + 1}'].append(nseqs_cluster / total_sequences * 100)
			alignment_info[f'Sequências Cluster {i + 1}'].append(nseqs_cluster)
		else:
			alignment_info[f'% Cluster {i + 1}'].append(0)
			alignment_info[f'Sequências Cluster {i + 1}'].append(0)


In [5]:
all_sequences = []
clusters_reports = [{'Amostra': [], 'Consenso': [], 'Comprimento': []} for i in range(clusters2process)]
for cluster_report_pos in range(len(clusters_reports)):
	for file, report in files_report.items():
		clusters_content = report['clusters_content']
		clusters_sizes = report['clusters_sizes']
		sequences = report['sequences']
		if cluster_report_pos < len(clusters_sizes):
			cluster = list(clusters_sizes.keys())[cluster_report_pos]
			
			cluster_content = clusters_content[cluster]
			scores = re.findall(r'>(.*)\.\.\. (.*)', cluster_content)
			sequences2align = []
			for score in scores:
				sequences2align.append(score[0])

			cluster_sequences = {item[0]: item[1] for item in sequences.items() if item[0] in sequences2align}
			# with open(f'./output/lib2/{file}/cluster{cluster_report_pos + 1}_sequences2align.fasta', 'w') as handle:
			# 	SeqIO.write(cluster_sequences.values(), handle, 'fasta')

			if len(cluster_sequences) > 2:
				# subprocess.run(
				# 	f'mafft --quiet --thread 10 ./output/lib2/{file}/cluster{cluster_report_pos + 1}_sequences2align.fasta > ./output/lib2/{file}/cluster{cluster_report_pos + 1}_alignment.fasta',
				# 	shell=True,
				# 	executable='/bin/bash'
				# )

				# subprocess.run(
				# 	f'source /home/bioinfo/miniconda3/etc/profile.d/conda.sh && \
				# 	conda activate emboss-env && \
				# 	cons ./output/lib2/{file}/cluster{cluster_report_pos + 1}_alignment.fasta ./output/lib2/{file}/cluster{cluster_report_pos + 1}_consensus.fasta && \
				# 	conda deactivate',
				# 	shell=True,
				# 	executable='/bin/bash'
				# )

				consensus_sequences = []
				with open(f'output/lib2/{file}/cluster{cluster_report_pos + 1}_consensus.fasta', 'r') as consensus_file:
					for record in SeqIO.parse(consensus_file, "fasta"):
						record.seq = Seq(str(record.seq).replace('\n', ''))
						record.seq = Seq(str(record.seq).replace('n', ''))
						record.seq = Seq(str(record.seq).replace('N', ''))
						record.id = f"cluster{cluster_report_pos + 1}@{file}"
						consensus_sequences.append(record)
						all_sequences.append(record)

						clusters_reports[cluster_report_pos]['Amostra'].append(file)
						clusters_reports[cluster_report_pos]['Consenso'].append(record.seq)
						clusters_reports[cluster_report_pos]['Comprimento'].append(len(record.seq))
						

				# with open(f'output/lib2/{file_name}/cluster{cluster_report_pos + 1}_consensus.fasta', 'w') as consensus_file:
				# 	SeqIO.write(consensus_sequences, consensus_file, "fasta")


In [6]:
from copy import deepcopy

clusters_reports_copy = deepcopy(clusters_reports)

In [161]:
with open(f'output/lib2/all_sequences.fasta', 'w') as file:
	SeqIO.write(all_sequences, file, "fasta")

In [7]:
subprocess.run(
	f'export BLASTDB=/media/bioinfo/6tb_hdd/04_Blast_Databases/taxdb && \
	blastn \
	-db /media/bioinfo/6tb_hdd/04_Blast_Databases/new_nt/nt \
	-query ./output/lib2/all_sequences.fasta \
	-outfmt "6 qseqid length score bitscore pident nident evalue gapopen gaps qcovs qcovhsp stitle sscinames mismatch qstart qend sstart send" \
	-out ./output/lib2/blast.tsv \
	-num_threads 6 \
	-max_target_seqs 10',
	shell=True,
	executable='/bin/bash'
)

CompletedProcess(args='export BLASTDB=/media/bioinfo/6tb_hdd/04_Blast_Databases/taxdb && \tblastn \t-db /media/bioinfo/6tb_hdd/04_Blast_Databases/new_nt/nt \t-query ./output/lib2/all_sequences.fasta \t-outfmt "6 qseqid length score bitscore pident nident evalue gapopen gaps qcovs qcovhsp stitle sscinames mismatch qstart qend sstart send" \t-out ./output/lib2/blast.tsv \t-num_threads 6 \t-max_target_seqs 10', returncode=0)

In [147]:
print(alignment_info)

{'Amostra': ['BAC-1.2', 'BAC-1.21', 'BAC-1.12'], 'Total de Sequências': [44, 2495, 1250], 'Clusters': [1, 3, 5], 'Sequências Cluster 1': [42, 2442, 753], '% Cluster 1': [95.45454545454545, 97.87575150300601, 60.24], 'Sequências Cluster 2': [0, 47, 396], '% Cluster 2': [0, 1.8837675350701404, 31.680000000000003], 'Sequências Cluster 3': [0, 6, 89], '% Cluster 3': [0, 0.24048096192384769, 7.12]}


In [8]:
alignment_info_df = pd.DataFrame(alignment_info)
blast_df = pd.read_csv('output/lib2/blast.tsv', sep='\t', names=['Amostra', 'length', 'score', 'bitscore', 'pident', 'nident', 'evalue', 'gapopen', 'gaps', 'qcovs', 'qcovhsp', 'stitle', 'sscinames', 'mismatch', 'qstart', 'qend', 'sstart', 'send'])
blast_df[['Cluster', 'Amostra']] = blast_df['Amostra'].str.split('@', n=1, expand=True)

In [9]:
def get_organism_taxid(organism, cursor):
	cursor.execute(f"SELECT * FROM names WHERE name_txt = '{organism}' LIMIT 1")
	result = cursor.fetchone()
	if result is None:
		cursor.execute(f"SELECT * FROM names WHERE MATCH(name_txt) AGAINST('{organism}' IN NATURAL LANGUAGE MODE)")
		result = cursor.fetchone()

	taxid = result['tax_id']
	name_txt = result['name_txt']

	return taxid, name_txt


def get_taxonomy(especie):
	# Connect to the database
	connection = pymysql.connect(
		host='localhost',
		user='root',
		password='Amora#1000',
		database='ncbi_data',
		cursorclass=pymysql.cursors.DictCursor

	)
	taxonomies = {'sscinames': None, 'Gênero': None, 'Família': None, 'Ordem': None, 'Classe': None, 'Filo': None, 'Super-Reino': None}
	with connection:
		with connection.cursor() as cursor:
			try:
				taxid, name_txt = get_organism_taxid(especie, cursor)
				
				cursor.execute(f"SELECT * FROM organisms WHERE tax_id = {taxid} LIMIT 1")
				# cursor.execute(f"SELECT * FROM organisms WHERE MATCH(tax_name) AGAINST('{especie}' IN NATURAL LANGUAGE MODE)")
				result = cursor.fetchone()

				taxonomies['sscinames'] = especie
				taxonomies['Gênero'] = result['genus']
				taxonomies['Família'] = result['family']
				taxonomies['Ordem'] = result['_order']
				taxonomies['Classe'] = result['class']
				taxonomies['Filo'] = result['phylum']
				taxonomies['Super-Reino'] = result['superkingdom']
				
			except Exception as e:				
				taxonomies['sscinames'] = especie
				taxonomies['Gênero'] = 'unclassified'
				taxonomies['Família'] = 'unclassified'
				taxonomies['Ordem'] = 'unclassified'
				taxonomies['Classe'] = 'unclassified'
				taxonomies['Filo'] = 'unclassified'
				taxonomies['Super-Reino'] = 'unclassified'
				
		
	return taxonomies

def build_excel_sheet(writer, sheet_name, df):
	df.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False, index=False)

	worksheet = writer.sheets[sheet_name]

	(max_row, max_col) = df.shape
	column_settings = [{'header': column} for column in df.columns]
	worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
	worksheet.set_column(0, max_col - 1, 15)

	return (writer, worksheet)

In [10]:
clusters_reports = [pd.DataFrame(cluster_report) for cluster_report in clusters_reports]

clusters_blast = blast_df.groupby('Cluster')
writer = pd.ExcelWriter(f'output/lib2/report.xlsx', engine='xlsxwriter')
writer, worksheet = build_excel_sheet(writer, f'INFO', alignment_info_df)

for group in clusters_blast.groups:
    cluster_blast_df = clusters_blast.get_group(group)
    cluster_pos = int(group.replace('cluster', '')) - 1
    samples_statistics = cluster_blast_df.groupby('Amostra')[['pident', 'qcovs', 'gaps']].mean().reset_index()
    samples_statistics.rename(columns={'pident': 'pident mean', 'qcovs': 'qcovs mean', 'gaps': 'gaps mean'}, inplace=True)

    most_frequent_organisms = cluster_blast_df.groupby('Amostra')['sscinames'].agg(pd.Series.mode).reset_index()
    most_frequent_organisms.rename(columns={'sscinames': 'sscinames mode'}, inplace=True)

    samples_statistics = samples_statistics.merge(most_frequent_organisms, on='Amostra')

    first_organisms = cluster_blast_df.groupby('Amostra').first().reset_index()
    first_organisms.rename(columns={'sscinames': 'Primeiro Resultado', 'pident': 'Primeiro Resultado pident', 'qcovs': 'Primeiro Resultado qcovs', 'gaps': 'Primeiro Resultado gaps'}, inplace=True)
    first_organisms = first_organisms[['Amostra', 'Primeiro Resultado', 'Primeiro Resultado pident', 'Primeiro Resultado qcovs', 'Primeiro Resultado gaps']]

    clusters_reports[cluster_pos] = clusters_reports[cluster_pos].merge(first_organisms, on='Amostra')
    clusters_reports[cluster_pos] = clusters_reports[cluster_pos].merge(samples_statistics, on='Amostra')

    unique_sscinames = cluster_blast_df['sscinames'].unique()

    with concurrent.futures.ThreadPoolExecutor(max_workers=24) as executor:
        futures = [executor.submit(get_taxonomy, ssciname) for ssciname in unique_sscinames]
        results = [future.result() for future in futures]
        taxonomies_df = pd.json_normalize(results)

    cluster_blast_df = cluster_blast_df.merge(taxonomies_df, on='sscinames', how='left').fillna('unclassified')


    cluster_blast_df = cluster_blast_df[['Amostra', 'pident', 'qcovs', 'gaps', 'stitle', 'sscinames', 'Gênero', 'Família', 'Ordem', 'Classe', 'Filo', 'Super-Reino', 'length', 'score', 'bitscore', 'nident', 'evalue', 'gapopen', 'qcovhsp', 'mismatch', 'qstart', 'qend', 'sstart', 'send']]

    writer, worksheet = build_excel_sheet(writer, f'BLAST Cluster {cluster_pos + 1}', cluster_blast_df)
    writer, worksheet = build_excel_sheet(writer, f'Consenso Cluster {cluster_pos + 1}', clusters_reports[cluster_pos])

writer.close()